In [61]:
import pandas as pd
import numpy as np
import tensorflow as tf
from imblearn.combine import SMOTETomek #Upsampling İşlemi İçin
from sklearn.model_selection import train_test_split,cross_val_score,RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,classification_report
import pickle
import warnings
warnings.filterwarnings("ignore")

In [62]:
#Datasetin okunması
df=pd.read_csv("heart_failure_clinical_records_dataset.csv") 
df.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,8,1


In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   age                       299 non-null    float64
 1   anaemia                   299 non-null    int64  
 2   creatinine_phosphokinase  299 non-null    int64  
 3   diabetes                  299 non-null    int64  
 4   ejection_fraction         299 non-null    int64  
 5   high_blood_pressure       299 non-null    int64  
 6   platelets                 299 non-null    float64
 7   serum_creatinine          299 non-null    float64
 8   serum_sodium              299 non-null    int64  
 9   smoking                   299 non-null    int64  
 10  time                      299 non-null    int64  
 11  DEATH_EVENT               299 non-null    int64  
dtypes: float64(3), int64(9)
memory usage: 28.2 KB


In [64]:
df.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.00000,285.000000,1.00000


In [65]:
# Null değer kontrolü
df.isnull().sum()

age                         0
anaemia                     0
creatinine_phosphokinase    0
diabetes                    0
ejection_fraction           0
high_blood_pressure         0
platelets                   0
serum_creatinine            0
serum_sodium                0
smoking                     0
time                        0
DEATH_EVENT                 0
dtype: int64

In [66]:
# unique değerler
df["DEATH_EVENT"].nunique()

2

In [67]:
# kaç tane veri var
print(f"veri sayısı: {df.shape[0]}")

# DEATH_EVENT bölümünde kaç tane sınıf var
print(f"sınıf sayısı: {df['DEATH_EVENT'].nunique()}")

# sınıfların dağılımı
print(f"sınıfların dağılımı: {df['DEATH_EVENT'].value_counts()}")

# sınıfların 100 de kaçlık dilimde olduğu
print(f"sınıfların 100 de kaçlık dilimde olduğu: {df['DEATH_EVENT'].value_counts(normalize=True)*100}")

veri sayısı: 299
sınıf sayısı: 2
sınıfların dağılımı: DEATH_EVENT
0    203
1     96
Name: count, dtype: int64
sınıfların 100 de kaçlık dilimde olduğu: DEATH_EVENT
0    67.892977
1    32.107023
Name: proportion, dtype: float64


In [68]:
X=df.drop("DEATH_EVENT",axis=1)
X

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,smoking,time
0,75.0,0,582,0,20,1,265000.00,1.9,130,0,4
1,55.0,0,7861,0,38,0,263358.03,1.1,136,0,6
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,7
3,50.0,1,111,0,20,0,210000.00,1.9,137,0,7
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,8
...,...,...,...,...,...,...,...,...,...,...,...
294,62.0,0,61,1,38,1,155000.00,1.1,143,1,270
295,55.0,0,1820,0,38,0,270000.00,1.2,139,0,271
296,45.0,0,2060,1,60,0,742000.00,0.8,138,0,278
297,45.0,0,2413,0,38,0,140000.00,1.4,140,1,280


In [69]:
y=df["DEATH_EVENT"]
y

0      1
1      1
2      1
3      1
4      1
      ..
294    0
295    0
296    0
297    0
298    0
Name: DEATH_EVENT, Length: 299, dtype: int64

In [70]:
y.value_counts()

DEATH_EVENT
0    203
1     96
Name: count, dtype: int64

### Upsampling

In [71]:
smote=SMOTETomek(random_state=42,n_jobs=-1)
X_,Y_=smote.fit_resample(X,y)

In [72]:
print(X_.shape,Y_.shape)

(322, 11) (322,)


In [73]:
Y_.value_counts()

DEATH_EVENT
1    161
0    161
Name: count, dtype: int64

### Model Oluşturulması

In [75]:
x_train,x_test,y_train,y_test=train_test_split(X_,Y_,test_size=0.25,random_state=42)


In [ ]:
x_train

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,smoking,time
168,47.000000,0,582,0,25,0,130000.000000,0.800000,134,0,201
148,45.000000,0,582,1,38,1,263358.030000,1.180000,137,0,185
221,45.000000,0,582,1,55,0,543000.000000,1.000000,132,0,250
227,63.000000,1,103,1,35,0,179000.000000,0.900000,136,1,270
139,65.000000,0,395,1,25,0,265000.000000,1.200000,136,1,154
...,...,...,...,...,...,...,...,...,...,...,...
188,64.000000,0,143,0,25,0,246000.000000,2.400000,135,0,214
71,75.000000,0,582,0,45,1,263358.030000,1.180000,137,0,87
106,61.000000,1,84,0,40,1,229000.000000,0.900000,141,0,110
270,57.507078,0,238,1,37,0,326507.077709,2.501416,138,0,29


In [ ]:
col=["anaemia","creatinine_phosphokinase","diabetes","ejection_fraction","high_blood_pressure","platelets","serum_creatinine","serum_sodium","smoking"]
for feature in col:
    x_train[feature].replace(0,x_train[feature].median(),inplace=True)

In [ ]:
rf=RandomForestClassifier(n_jobs=-1)
rf.fit(x_train,y_train)

RandomForestClassifier(n_jobs=-1)

In [ ]:
score=cross_val_score(rf,x_train,y_train,cv=10,n_jobs=-1)

In [ ]:
##Model training accuracy
score

array([0.88      , 0.91666667, 0.83333333, 1.        , 0.91666667,
       1.        , 0.91666667, 0.75      , 0.91666667, 0.75      ])

In [ ]:
score.shape

(10,)

In [ ]:
score.mean()

0.8879999999999999

### Mean training accuracy  88.8%

In [ ]:
score_test=cross_val_score(rf,x_test,y_test,cv=10,n_jobs=-1)
score_test

array([0.88888889, 0.75      , 0.75      , 1.        , 0.625     ,
       0.75      , 0.75      , 0.625     , 1.        , 0.75      ])

In [ ]:
score_test.mean()

0.788888888888889

### Mean test accuracy is 82.73 %

In [ ]:
print("The maximum accuracy that our model can get is {} and minimum accuracy the model can get is {}".format(np.round(score_test.max(),2),np.round(score_test.min(),2)))

The maximum accuracy that our model can get is 1.0 and minimum accuracy the model can get is 0.62


In [ ]:
pred=rf.predict(x_test)

In [ ]:
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

           0       0.92      0.86      0.89        42
           1       0.86      0.92      0.89        39

    accuracy                           0.89        81
   macro avg       0.89      0.89      0.89        81
weighted avg       0.89      0.89      0.89        81



In [ ]:
##hyperparmeters of random forest
RandomForestClassifier()

RandomForestClassifier()

## Hyperparmeter Tunning

In [ ]:
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt','log2']
max_depth = [int(x) for x in np.linspace(10, 1000,10)]
min_samples_split = [2, 5, 10,14]
min_samples_leaf = [1, 2, 4,6,8]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
              'criterion':['entropy','gini']}
print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [10, 120, 230, 340, 450, 560, 670, 780, 890, 1000], 'min_samples_split': [2, 5, 10, 14], 'min_samples_leaf': [1, 2, 4, 6, 8], 'criterion': ['entropy', 'gini']}


In [78]:
rs=RandomizedSearchCV(rf,random_grid,cv=6,verbose=2,n_jobs=-1)
rs.fit(x_train,y_train)

Fitting 6 folds for each of 10 candidates, totalling 60 fits


RandomizedSearchCV(cv=6, estimator=RandomForestClassifier(n_jobs=-1), n_jobs=-1,
                   param_distributions={'criterion': ['entropy', 'gini'],
                                        'max_depth': [10, 120, 230, 340, 450,
                                                      560, 670, 780, 890,
                                                      1000],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': [1, 2, 4, 6, 8],
                                        'min_samples_split': [2, 5, 10, 14],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   verbose=2)

In [ ]:
pred=rs.predict(x_test)


In [ ]:
print(classification_report(pred,y_test))

              precision    recall  f1-score   support

           0       0.81      0.92      0.86        37
           1       0.92      0.82      0.87        44

    accuracy                           0.86        81
   macro avg       0.87      0.87      0.86        81
weighted avg       0.87      0.86      0.86        81



In [ ]:
model_file=open("model.h5","wb")
pickle.dump(rs,model_file)
model_file.close()

In [ ]:
model=pickle.load(open("model.h5","rb"))